In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [21]:
df = pd.read_csv('../Data/dataset_regression.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.isnull().sum().sum()

0

In [22]:
y = df["price_usd"]
X = df.drop(["price_usd"], axis=1)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

In [24]:
# создаем модель, как набор последовательных слоев
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_shape=(X.shape[1],)),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [25]:
model_regression.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                1920      
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 4,545
Trainable params: 4,545
Non-traina

In [26]:
# компилируем
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [27]:
model_regression.fit(X_train, y_train, epochs=50)

Epoch 1/50
803/803 [==============================] - 2s 1ms/step - loss: 56606208.0000
Epoch 2/50
803/803 [==============================] - 1s 1ms/step - loss: 34121336.0000
Epoch 3/50
803/803 [==============================] - 1s 1ms/step - loss: 31661562.0000
Epoch 4/50
803/803 [==============================] - 1s 1ms/step - loss: 31011292.0000
Epoch 5/50
803/803 [==============================] - 1s 1ms/step - loss: 29692146.0000
Epoch 6/50
803/803 [==============================] - 1s 1ms/step - loss: 29618404.0000
Epoch 7/50
803/803 [==============================] - 1s 1ms/step - loss: 29080184.0000
Epoch 8/50
803/803 [==============================] - 1s 1ms/step - loss: 28334536.0000
Epoch 9/50
803/803 [==============================] - 1s 1ms/step - loss: 27826488.0000
Epoch 10/50
803/803 [==============================] - 1s 1ms/step - loss: 27557710.0000
Epoch 11/50
803/803 [==============================] - 1s 1ms/step - loss: 27039516.0000
Epoch 12/50
803/803 [=========

In [28]:
# оцениваем качество с помощью метрик
print(mean_absolute_error(y_test, model_regression.predict(X_test)))
print(mean_squared_error(y_test, model_regression.predict(X_test)))

402/402 [==============================] - 0s 750us/step
2560.952003808035
402/402 [==============================] - 0s 882us/step
13876407.405323


In [51]:
pred_test = model_regression.predict(X_test)
pred_test

402/402 [==============================] - 0s 782us/step


array([[ 7630.582 ],
       [ 2371.7097],
       [11661.832 ],
       ...,
       [ 6220.571 ],
       [ 2633.95  ],
       [10133.862 ]], dtype=float32)

In [64]:
pred_pred = pred_test.ravel()
pred_pred

array([ 7630.582 ,  2371.7097, 11661.832 , ...,  6220.571 ,  2633.95  ,
       10133.862 ], dtype=float32)

In [66]:
def MSE(predicted, real):
    return np.sum((real-predicted)**2)/len(real)

In [67]:
def R2(predicted, real):
    return 1 - ((MSE(predicted, real) * np.mean(real)) / np.sum((real - predicted)**2))

In [68]:
R2(pred_pred, y_test)

0.4858537882081808